In [7]:
import numpy as np

In [8]:
def natural_cubic_spline_coeffs(x, y):
    """
    Given arrays x[0..n], y[0..n], construct the 'natural' cubic spline
    and return the second derivative values M[0..n] at the knots.
    """
    n = len(x) - 1  
    h = x[1:] - x[:-1]

    A = np.zeros((n-1, n-1))
    rhs = np.zeros(n-1)
    
    # Tridiagonal system for M_i
    for i in range(1, n):
        if i == 1:
            A[i-1, i-1] = 2*(h[i-1] + h[i])  # diagonal
            if n-1 > 1:  # if there is more than one interior point
                A[i-1, i] = h[i]            # superdiagonal
            rhs[i-1] = 6.0 * ((y[i+1] - y[i]) / h[i] - (y[i] - y[i-1]) / h[i-1])
        elif i == n-1:
            A[i-1, i-2] = h[i-1]           # subdiagonal
            A[i-1, i-1] = 2*(h[i-1] + h[i]) 
            rhs[i-1] = 6.0 * ((y[i+1] - y[i]) / h[i] - (y[i] - y[i-1]) / h[i-1])
        else:
            # subdiagonal
            A[i-1, i-2] = h[i-1]
            # diagonal
            A[i-1, i-1] = 2*(h[i-1] + h[i])
            # superdiagonal
            A[i-1, i] = h[i]
            rhs[i-1] = 6.0 * ((y[i+1] - y[i]) / h[i] - (y[i] - y[i-1]) / h[i-1])
    
    # Solve for M[1..n-1]
    M_interior = np.linalg.solve(A, rhs) if n > 1 else np.array([])
    
    # Construct full M array
    M = np.zeros(n+1)
    if n > 1:
        M[1:n] = M_interior
    
    return M

In [9]:
def f(x):
    return np.cos(2.0 * np.pi * x)

def fprime(x):
    return -2.0 * np.pi * np.sin(2.0 * np.pi * x)

def fprime2(x):
    return -4.0 * (np.pi**2) * np.cos(2.0 * np.pi * x)

In [10]:
def spline_eval(x_data, y_data, M, x_query):
    """
    Evaluate the natural cubic spline s(x) at a single point x_query
    """
    # Find interval i s.t. x_query in [x_data[i], x_data[i+1]]
    i = np.searchsorted(x_data, x_query) - 1
    i = max(0, min(i, len(x_data)-2))  # clamp
    
    h = x_data[i+1] - x_data[i]
    xi, xi1 = x_data[i], x_data[i+1]
    yi, yi1 = y_data[i], y_data[i+1]
    Mi, Mi1 = M[i], M[i+1]
    
    A = (xi1 - x_query) / h
    B = (x_query - xi) / h
    s_val = ( (A**3 - A) * Mi + (B**3 - B) * Mi1 ) * (h**2)/6.0 \
            + A * yi + B * yi1
    
    return s_val

def spline_deriv1(x_data, y_data, M, x_query):
    """
    Evaluate the first derivative s'(x_query) of the natural cubic spline.
    """
    i = np.searchsorted(x_data, x_query) - 1
    i = max(0, min(i, len(x_data)-2))
    
    h = x_data[i+1] - x_data[i]
    xi, xi1 = x_data[i], x_data[i+1]
    yi, yi1 = y_data[i], y_data[i+1]
    Mi, Mi1 = M[i], M[i+1]
    
    A = (xi1 - x_query) / h
    B = (x_query - xi) / h

    termA = - ((xi1 - x_query)**2) / (2.0 * h) * Mi
    termB =   ((x_query - xi)**2) / (2.0 * h) * Mi1
    termC = (yi/h - (Mi * h)/6.0) * (-1.0)
    termD = (yi1/h - (Mi1 * h)/6.0) * (+1.0)
    
    return termA + termB + termC + termD

def spline_deriv2(x_data, M, x_query):
    """
    Evaluate the second derivative s''(x_query) of the natural cubic spline.
    """
    i = np.searchsorted(x_data, x_query) - 1
    i = max(0, min(i, len(x_data)-2))
    
    h = x_data[i+1] - x_data[i]
    xi, xi1 = x_data[i], x_data[i+1]
    Mi, Mi1 = M[i], M[i+1]
    
    A = (xi1 - x_query) / h
    B = (x_query - xi) / h

    return A * Mi + B * Mi1

In [ ]:
def finite_diff_deriv1(x, y):
    """
    Approximate first derivatives at the nodes x[i] using central differences.
    f'(x_i) ~ (f(x_{i+1}) - f(x_{i-1})) / (2h).
    For the endpoints, we'll do a one-sided approximation.
    """
    n = len(x)
    df = np.zeros(n)
    h = x[1] - x[0]  
    
    # Forward difference at i=0
    df[0] = (y[1] - y[0]) / h
    # Central differences for i=1..n-2
    for i in range(1, n-1):
        df[i] = (y[i+1] - y[i-1]) / (2.0 * h)
    # Backward difference at i=n-1
    df[n-1] = (y[n-1] - y[n-2]) / h
    
    return df

def finite_diff_deriv2(x, y):
    """
    Approximate second derivatives at the nodes x[i] using central differences:
    f''(x_i) ~ ( f(x_{i+1}) - 2 f(x_i) + f(x_{i-1}) ) / h^2
    For endpoints, do a lower-order approximation.
    """
    n = len(x)
    d2f = np.zeros(n)
    h = x[1] - x[0]  
    
    if n >= 3:
        d2f[0] = (y[2] - 2*y[1] + y[0]) / (h*h)
    else:
        d2f[0] = 0.0  
    
    # Central differences for i=1..n-2
    for i in range(1, n-1):
        d2f[i] = (y[i+1] - 2*y[i] + y[i-1]) / (h*h)
    
    # For i=n-1, do a backward difference approximation if n>=3
    if n >= 3:
        d2f[n-1] = (y[n-1] - 2*y[n-2] + y[n-3]) / (h*h)
    else:
        d2f[n-1] = 0.0
    
    return d2f

In [12]:
n = 5  
x_data = np.linspace(0.0, 1.0, n+1)  
y_data = f(x_data)

M = natural_cubic_spline_coeffs(x_data, y_data)

spline_d1 = np.array([spline_deriv1(x_data, y_data, M, x) for x in x_data])
spline_d2 = np.array([spline_deriv2(x_data, M, x) for x in x_data])

exact_d1 = fprime(x_data)
exact_d2 = fprime2(x_data)

fd_d1 = finite_diff_deriv1(x_data, y_data)
fd_d2 = finite_diff_deriv2(x_data, y_data)

print(" i   x_i      f'(x_i) [exact]     s'(x_i) [spline]   f'(x_i) [FD]      Error spline   Error FD")
for i in range(n+1):
    err_spline = abs(spline_d1[i] - exact_d1[i])
    err_fd     = abs(fd_d1[i] - exact_d1[i])
    print(f"{i:2d}  {x_data[i]:7.4f}   {exact_d1[i]:15.8f}   {spline_d1[i]:15.8f}   {fd_d1[i]:15.8f}"
            f"   {err_spline:12.4e}   {err_fd:12.4e}")

print("\n i   x_i      f''(x_i) [exact]    s''(x_i) [spline]  f''(x_i) [FD]     Error spline   Error FD")
for i in range(n+1):
    err_spline = abs(spline_d2[i] - exact_d2[i])
    err_fd     = abs(fd_d2[i] - exact_d2[i])
    print(f"{i:2d}  {x_data[i]:7.4f}   {exact_d2[i]:15.8f}   {spline_d2[i]:15.8f}   {fd_d2[i]:15.8f}"
            f"   {err_spline:12.4e}   {err_fd:12.4e}")


 i   x_i      f'(x_i) [exact]     s'(x_i) [spline]   f'(x_i) [FD]      Error spline   Error FD
 0   0.0000       -0.00000000       -2.59878637       -3.45491503     2.5988e+00     3.4549e+00
 1   0.2000       -5.97566433       -5.16717235       -4.52254249     8.0849e-01     1.4531e+00
 2   0.4000       -3.69316366       -3.86777916       -2.79508497     1.7462e-01     8.9808e-01
 3   0.6000        3.69316366        3.86777916        2.79508497     1.7462e-01     8.9808e-01
 4   0.8000        5.97566433        5.16717235        4.52254249     8.0849e-01     1.4531e+00
 5   1.0000        0.00000000        2.59878637        3.45491503     2.5988e+00     3.4549e+00

 i   x_i      f''(x_i) [exact]    s''(x_i) [spline]  f''(x_i) [FD]     Error spline   Error FD
 0   0.0000      -39.47841760        0.00000000      -10.67627458     3.9478e+01     2.8802e+01
 1   0.2000      -12.19950195      -25.68385977      -10.67627458     1.3484e+01     1.5232e+00
 2   0.4000       31.93871075       38.67

Discussion:
-----------
1) The spline-derived first derivatives s'(x_i) and second derivatives s''(x_i)
   usually provide higher accuracy at the nodes than simple finite-difference formulas.
2) The natural spline imposes M_0 = M_n = 0, which can slightly bias
   the endpoints if the function’s second derivative is nonzero there.
3) Finite difference approximations degrade near boundaries
   (due to one-sided differences) and are generally lower-order
   than the piecewise cubic spline derivative at the nodes.
4) This example shows that spline-based differentiation is often more
   accurate for smooth functions like cos(2πx) than naive finite differences.